In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2
import numpy as np
import glob
import os

# フォルダの指定(sampleフォルダ)
folder_path = 'drive/MyDrive/Colab Notebooks/dataset/sample/*.png'

# 指定フォルダの探索（画像を探す）
pic_list = glob.glob(folder_path)

if not pic_list:
    print("No images found in the specified folder.")
else:
    # リサイズ・平均化した画像を保存するフォルダの指定（フォルダは自動で作成されます）
    output_folder = 'drive/MyDrive/Colab Notebooks/resize_ave/'
    os.makedirs(output_folder, exist_ok=True)

    # リサイズ・平均化した画像のサイズを指定
    common_size = (256, 256)

    # sampleフォルダ内の画像を個別処理する
    for i, pic_single in enumerate(pic_list, start=1):
        # 画像の読み込み
        img_single = cv2.imread(pic_single)

        # 画像のリサイズ
        img_single = cv2.resize(img_single, common_size)

        # リサイズした画像の変数の初期化
        base_array = np.zeros((common_size[1], common_size[0], 3), np.float32)
        total_weight = 0

        # 画像の平均化処理
        for pic_reference in pic_list:
            img_reference = cv2.imread(pic_reference)
            img_reference = cv2.resize(img_reference, common_size)

            if np.array_equal(img_single, img_reference):
                # 平均化した画像の評価
                sharpness = cv2.Laplacian(img_reference, cv2.CV_64F).var()

                # 重み
                weight = sharpness if sharpness > 1000 else 0.8

                # 重み付けした画像のピクセル値を累積
                base_array = base_array + img_reference * weight
                total_weight += weight

                print(f"Sharpness of {pic_reference}: {sharpness}")

        if total_weight > 0:
            # ピクセル値の正規化
            base_array = (base_array / total_weight).astype(np.uint8)

            # ファイル名を指定して保存
            output_filename = os.path.join(output_folder, f'resize_ave_{i:02d}.png')
            cv2.imwrite(output_filename, base_array)

            print(f"Individual average image saved to {output_filename}")
        else:
            print(f"No matching instances found for {pic_single}")
